# Import modules 

In [94]:
import pandas as pd
import qgrid
import easygui
import glob
import math

# Helper functions

In [102]:
def count_digits(input):
    return int(math.log10(input))+1

def ID_padder(input):
    current_year = 63
    if count_digits(input) == 8:
        return current_year*10**8 + input
    elif count_digits(input) == 9:
        return 6*10**9 + input
    elif count_digits(input) == 10:
        return input
    else:
        raise Exception(str(input) + " -> ID Year out of range")

# Import data

In [125]:
# import data

default_dir = "/Users/tanwaarpornthip/src/teaching/cpr/2020-2"
print("Select a file inside a folder that contains the raw grade file.")
target_dir = easygui.diropenbox("Select a folder that containts the raw check-in files", 
  "Select a folder that containts the raw check-in files",default=default_dir)
file_data = []
for file in glob.glob(target_dir+"/Log*.txt"):
    df = pd.read_csv(file,skiprows=1,sep='\t')
    file_data.append(df)

replacement_card_data = pd.read_csv('replacement.txt', header=None)

Select a file inside a folder that contains the raw grade file.


In [126]:
data = pd.concat(file_data)
data = data.drop(["No","Mchn","Name", "Unnamed: 5", "IOMd","DateTime","Unnamed: 9"],axis=1)
data = data.rename(columns={"Unnamed: 3": "Name", "EnNo":"ID"})

In [134]:
# Get replacement card data
replacement = replacement_card_data.set_index(0).to_dict(orient='dict')[1]
# Replacing temp card with replacement data
data['ID'].replace(replacement, inplace=True)
# Padding student IDs
data['ID_length']=data['ID'].apply(count_digits)
data['full_ID']=data['ID'].apply(ID_padder)

In [136]:
data.to_excel('test.xlsx')